In [1]:
import polars as pl
from op_datasets.processing.execute import reader, BatchInput
from op_datasets.logic.transforms.daily_address_summary import (
    daily_address_summary,
    CONDITIONS,
)
from op_coreutils.logger import structlog
from op_coreutils.bigquery.write import overwrite_partitions

from datetime import datetime


log = structlog.get_logger()

2024-10-16 19:31:00 [info     ] Loaded env var: OP_ANALYTICS_VAULT


In [2]:
inputbatch: BatchInput = None
for _inputbatch in reader(
    chain="op", block_spec="105376883:+500", source_spec="goldsky"
):
    inputbatch = _inputbatch
    break

dataframes = inputbatch.dataframes

2024-10-16 19:31:01 [info     ] Loaded 5 items into vault.    
2024-10-16 19:31:02 [info     ] Initialized Clickhouse client.
2024-10-16 19:31:02 [info     ] Query success: blocks in 1.70s 2000 rows
2024-10-16 19:31:03 [info     ] Query success: transactions in 2.49s 14166 rows
2024-10-16 19:31:03 [info     ] Query success: logs in 2.75s 59381 rows


In [3]:
dataframes

{'blocks': shape: (2_000, 24)
 ┌───────┬─────────┬──────────┬────────────┬───┬──────────┬───────────┬────────────┬────────────────┐
 │ chain ┆ network ┆ chain_id ┆ dt         ┆ … ┆ gas_used ┆ gas_limit ┆ extra_data ┆ transaction_co │
 │ ---   ┆ ---     ┆ ---      ┆ ---        ┆   ┆ ---      ┆ ---       ┆ ---        ┆ unt            │
 │ str   ┆ str     ┆ i32      ┆ str        ┆   ┆ i64      ┆ i64       ┆ str        ┆ ---            │
 │       ┆         ┆          ┆            ┆   ┆          ┆           ┆            ┆ i64            │
 ╞═══════╪═════════╪══════════╪════════════╪═══╪══════════╪═══════════╪════════════╪════════════════╡
 │ op    ┆ mainnet ┆ 10       ┆ 2023-06-09 ┆ … ┆ 676868   ┆ 30000000  ┆ 0x         ┆ 5              │
 │ op    ┆ mainnet ┆ 10       ┆ 2023-06-09 ┆ … ┆ 1149120  ┆ 30000000  ┆ 0x         ┆ 7              │
 │ op    ┆ mainnet ┆ 10       ┆ 2023-06-09 ┆ … ┆ 3368746  ┆ 30000000  ┆ 0x         ┆ 9              │
 │ op    ┆ mainnet ┆ 10       ┆ 2023-06-09 ┆ … ┆ 417

In [4]:
# check if our blocks are in the range we want

log.info(f"Min block number: {dataframes['blocks']['number'].min()}")
log.info(f"Max block number: {dataframes['blocks']['number'].max()}")
log.info(
    f"Min block number: {datetime.fromtimestamp(dataframes['blocks']['timestamp'].min())}"
)
log.info(
    f"Max block number: {datetime.fromtimestamp(dataframes['blocks']['timestamp'].max())}"
)

2024-10-16 19:31:03 [info     ] Min block number: 105376000   
2024-10-16 19:31:03 [info     ] Max block number: 105377999   
2024-10-16 19:31:03 [info     ] Min block number: 2023-06-09 18:46:17
2024-10-16 19:31:03 [info     ] Max block number: 2023-06-09 19:52:55


In [5]:
result = daily_address_summary(
    dataframes["transactions"],
    conditions=CONDITIONS,
    address="from_address",
    chain_id="chain_id",
    chain="chain",
    dt="dt",
)

In [6]:
# list(result.head()["hash"])
result.head()

from_address,chain_id,chain,dt,total_txs,total_txs_success,total_txs_fail,total_blocks,total_blocks_success,total_blocks_fail,min_block_number,max_block_number,block_interval_active,min_nonce,max_nonce,nonce_interval_active,total_l2_gas_used,total_l2_gas_used_success,total_l2_gas_used_fail,total_l1_gas_used,total_l1_gas_used_success,total_l1_gas_used_fail,total_gas_fees,total_gas_fees_success,total_gas_fees_fail,l2_contrib_gas_fees,l1_contrib_gas_fees,l1_blobgas_contrib_gas_fees,l1_l1gas_contrib_gas_fees,l2_contrib_gas_fees_base_fee,l2_contrib_gas_fees_priority_fee,avg_l2_gas_price_gwei,avg_l2_base_fee_gwei,avg_l2_priority_fee_gwei,avg_l1_gas_price_gwei,avg_l1_blob_base_fee_gwei,min_block_timestamp,max_block_timestamp,time_interval_active,unique_hours_active,perc_daily_hours_active,num_to_addresses,num_to_addresses_success,num_to_addresses_fail
str,i32,str,str,u32,u32,u32,u32,u32,u32,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,"decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,17]","decimal[*,17]","decimal[*,17]","decimal[*,17]","decimal[*,17]",datetime[μs],datetime[μs],duration[μs],u32,f64,u32,u32,u32
"""0xadfa6ef1d6223225feb30478601f…",10,"""op""","""2023-06-09""",11,11,0,11,11,0,105377298,105377964,667,256,266,11,2883614,2883614,0,43260,43260,0,0.0005303079217957950000,0.0005303079217957950000,0.0000000000000000000000,0.0000001551734823450000,0.0005301527483134500000,0.0000000000000000000000,0.0005301527483134551274,0.0000000001626872770000,0.0000001550107950680000,0.00005381215458969,0.00000005641784129,0.00005375573674839,17.91671561297577438,0.00000000000000000,2023-06-09 23:29:33,2023-06-09 23:51:45,22m 12s,1,0.041667,9,9,0
"""0x10c91ca7d72c32ccc83ca9fdd1b1…",10,"""op""","""2023-06-09""",1,1,0,1,1,0,105377873,105377873,1,106,106,1,48847,48847,0,2100,2100,0,0.0000244624181938840000,0.0000244624181938840000,0.0000000000000000000000,0.0000000000316528560000,0.0000244623865410280000,0.0000000000000000000000,0.0000244623865410287996,0.0000000000000000000000,0.0000000000316528560000,0.00000064800000000,0.00000000000000000,0.00000064800000000,17.03034429200000000,0.00000000000000000,2023-06-09 23:48:43,2023-06-09 23:48:43,0µs,1,0.041667,1,1,0
"""0x843ca717461a5d27ec57d26e0761…",10,"""op""","""2023-06-09""",4,4,0,4,4,0,105376256,105376272,17,406,409,4,133482,133482,0,9404,9404,0,0.0001110753694999260000,0.0001110753694999260000,0.0000000000000000000000,0.0000000158450000000000,0.0001110595244999260000,0.0000000000000000000000,0.0001110595244999281979,0.0000000158450000000000,0.0000000000000000000000,0.00011870514376470,0.00011870514376470,0.00000000000000000,17.26581517195746490,0.00000000000000000,2023-06-09 22:54:49,2023-06-09 22:55:21,32s,1,0.041667,4,4,0
"""0x983cf3749289b92366204899b457…",10,"""op""","""2023-06-09""",1,1,0,1,1,0,105376598,105376598,1,8636,8636,1,362569,362569,0,6984,6984,0,0.0000769151293885490000,0.0000769151293885490000,0.0000000000000000000000,0.0000000000253798300000,0.0000769151040087190000,0.0000000000000000000000,0.0000769151040087194045,0.0000000000217541400000,0.0000000000036256900000,0.00000007000000000,0.00000006000000000,0.00000001000000000,16.10094250700000000,0.00000000000000000,2023-06-09 23:06:13,2023-06-09 23:06:13,0µs,1,0.041667,1,1,0
"""0x1ca2d60c2329999bc824553eee6c…",10,"""op""","""2023-06-09""",1,1,0,1,1,0,105376699,105376699,1,91,91,1,283024,283024,0,2524,2524,0,0.0000319762704991730000,0.0000319762704991730000,0.0000000000000000000000,0.0000000000605671360000,0.0000319762099320370000,0.0000000000000000000000,0.0000319762099320379237,0.0000000000605671360000,0.0000000000000000000000,0.00000021400000000,0.00000021400000000,0.00000000000000000,18.52172937000000000,0.00000000000000000,2023-06-09 23:09:35,2023-06-09 23:09:35,0µs,1,0.041667,1,1,0


In [7]:
overwrite_partitions(result, "temp", "daily_address_summary", expiration_days=999)

2024-10-16 19:31:04 [info     ] Writing 1 partitions to BQ [2023-06-09 00:00:00 ... 2023-06-09 00:00:00]
2024-10-16 19:31:04 [info     ] DRYRUN OVERWRITE PARTITION: Wrote 2.9Krows 390.9KB to BQ temp.daily_address_summary
